In [22]:
import pandas as pd
import numpy as np
from siuba import *
from calitp import *

In [23]:
import altair as alt

In [24]:
from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [25]:
pd.set_option("display.precision", 2)

In [26]:
df = pd.read_excel('grants_overview_walkthrough.xlsx')

In [27]:
df = to_snakecase(df)

In [28]:
df = df.rename(columns={'grant_type__formula,_competitive,_both,_discretionary_n_a_':'distribution_type',
                       'current__in_this_fy__or_expired':'current'})

In [29]:
df.sample()

,ct_division,grant_program,fy_due_date,fy_cycle,funding_amount_available,availibility,agency_eligibility,eligibility_restrictions,federal_or_state_funded,distribution_type,...,eligibility:_municipality,eligibility:_school_district,eligibility:_mpo,eligibility:_county_regional_govt__not_mpo,eligibility:_other_local_entity__non_municipality_,eligibility:_state_public_lands,eligibility:_federal_public_lands,eligibility:_tribal_government,eligibility:_non_profit,eligibility:_state_department_
29,DLA,Emergency Relief Program,"14 days, 3m, 10y after disaster",rolling,1.00e+08,state,"A California City, county, tribal government o...",NaN,Federal,Discretionary,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN


In [30]:
df.availibility.value_counts()

state         25
nationwide     6
Name: availibility, dtype: int64

In [31]:
(
    df
    >>filter(_.availibility=='state')
    >> group_by(_.ct_division, _.federal_or_state_funded)
    >> summarize(avg_funds=(_.funding_amount_available.mean()))
)

,ct_division,federal_or_state_funded,avg_funds
0,Clean CA,State,2.96e+08
1,DLA,Federal,1.30e+08
2,DLA,State,1.27e+07
3,DRMT,Both,5.00e+08
4,DRMT,State,9.16e+07
5,SB1,Both,2.56e+09
6,SB1,Federal,1.88e+07
7,SB1,State,4.78e+08
8,SB1/DLA,State,5.79e+08


In [32]:
chart = (
    alt.Chart(
        df
        >>filter(_.availibility=='state')
        >> group_by(_.ct_division, _.federal_or_state_funded)
        >> summarize(avg_funds=(_.funding_amount_available.mean()))
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds", axis=alt.Axis(format="$.2s"), title="Average Funds Available"
        ),
        y=alt.Y("ct_division", title="Division"),
        color=alt.Color(
            "federal_or_state_funded",
            title="Origin of Funding",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
        ),
    )
    .properties(title="Average Funds Available by Type")
)
styleguide.preset_chart_config(chart)

alt.Chart(...)

In [33]:
chart = (
    alt.Chart(
        df
        >>filter(_.availibility=='nationwide')
        >> group_by(_.ct_division, _.federal_or_state_funded)
        >> summarize(avg_funds=(_.funding_amount_available.mean()))
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds", axis=alt.Axis(format="$.2s"), title="Average Funds Available"
        ),
        y=alt.Y("ct_division", title="Division"),
        color=alt.Color(
            "federal_or_state_funded",
            title="Origin of Funding",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
        ),
    )
    .properties(title="Average Funds Available by Type (Nation-wide)")
)
styleguide.preset_chart_config(chart)

alt.Chart(...)

In [35]:
df>>group_by(_.ct_division)>>summarize(avg_funds = (_.funding_amount_available.mean()))

,ct_division,avg_funds
0,Clean CA,2.96e+08
1,DLA,8.30e+07
2,DLA,NaN
3,DRMT,1.41e+09
4,SB1,8.56e+08
5,SB1/DLA,5.79e+08


In [36]:
chart = (
    alt.Chart(
        (
            df
             >>filter(_.availibility=='state')
            >> group_by(_.ct_division)
            >> summarize(avg_funds=(_.funding_amount_available.mean()))
        )
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds", axis=alt.Axis(format="$.2s"), title="Average Funds Available"
        ),
        y=alt.Y("ct_division", title="Division"),
        color=alt.Color(
            "ct_division",
            title="Division",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
        ),
    )
    .properties(title="Average Funds Available by Division")
)
styleguide.preset_chart_config(chart)

alt.Chart(...)

In [42]:
chart = (
    alt.Chart(
        (
            df
             >>filter(_.availibility=='nationwide')
            >> group_by(_.ct_division)
            >> summarize(avg_funds=(_.funding_amount_available.mean()))
        )
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds", axis=alt.Axis(format="$.2s"), title="Average Funds Available"
        ),
        y=alt.Y("ct_division", title="Division"),
        color=alt.Color(
            "ct_division",
            title="Division",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
        ),
    )
    .properties(title="Average Funds Available by Division (Nation-wide)")
)
styleguide.preset_chart_config(chart)

alt.Chart(...)

In [37]:
df>>group_by(_.distribution_type, _.ct_division)>>summarize(avg_funds = (_.funding_amount_available.mean()))

,distribution_type,ct_division,avg_funds
0,Both,DRMT,6.04e+08
1,Both,SB1,1.17e+08
2,Competitive,DLA,1.77e+07
3,Competitive,DRMT,2.40e+09
4,Competitive,SB1,1.72e+09
5,Competitive,Clean CA,2.96e+08
6,Competitive,DRMT,2.28e+08
7,Competitive,SB1,7.25e+08
8,Discretionary,DLA,1.00e+08
9,Formula,DLA,1.04e+08


In [39]:
chart = (
    alt.Chart(
        (
            df
             >>filter(_.availibility=='state')
            >> group_by(_.distribution_type, _.ct_division)
            >> summarize(avg_funds=(_.funding_amount_available.mean()))
        )
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds",
            axis=alt.Axis(format="$.2s"),
            title=("Average Funds Available"),
            sort=("-y"),
        ),
        y=alt.Y("ct_division", title="Division"),
        color=alt.Color(
            "distribution_type",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
            legend=alt.Legend(title=("Distribution Type")),
        ),
    )
    .properties(title="Average Funds Available by Disvision and Distribution Type")
)
chart = styleguide.preset_chart_config(chart)
chart

alt.Chart(...)

In [41]:
chart = (
    alt.Chart(
        (
            df
             >>filter(_.availibility=='nationwide')
            >> group_by(_.distribution_type, _.ct_division)
            >> summarize(avg_funds=(_.funding_amount_available.mean()))
        )
    )
    .mark_bar()
    .encode(
        x=alt.X(
            "avg_funds",
            axis=alt.Axis(format="$.2s"),
            title=("Average Funds Available"),
            sort=("-y"),
        ),
        y=alt.Y("ct_division", title="Division"),
        color=alt.Color(
            "distribution_type",
            scale=alt.Scale(range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
            legend=alt.Legend(title=("Distribution Type")),
        ),
    )
    .properties(title="Average Funds Available by Disvision and Distribution Type (Nation-wide)")
)
chart = styleguide.preset_chart_config(chart)
chart

alt.Chart(...)